In [88]:
import numpy as np

def calculate_element_local_stiffness_matrix(E, A, L):
    ke_local = np.zeros((4,4))
    ke_local[0, 0] = E * A / L
    ke_local[0, 2] = -E * A / L
    ke_local[2, 0] = -E * A / L
    ke_local[2, 2] = E * A / L
    return ke_local

def define_element_rotation_matrix(theta):
    beta = np.zeros((4, 4))
    unit_rotation = np.array([[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]])
    beta[0:2, 0:2] = unit_rotation
    beta[2:4, 2:4] = unit_rotation
    return beta

def calculate_element_length(node_i_coordinates, node_j_coordinates):
    L = np.linalg.norm(node_j_coordinates - node_i_coordinates)
    return L

def calculate_element_orientation(node_i_coordinates, node_j_coordinates):
    delta = node_j_coordinates - node_i_coordinates
    theta = np.arctan2(delta[1], delta[0])
    return theta

def define_global_element_stiffness_matrix(ke_local, beta):
    ke_global = np.dot(np.dot(beta.T, ke_local), beta)
    return ke_global

def define_global_dof_for_element(node_number_i, node_number_j):
    dof_i = [node_number_i * 2 - 1, node_number_i * 2]
    dof_j = [node_number_j * 2 - 1, node_number_j * 2]
    dof = np.array([dof_i, dof_j])
    return dof

def assemble_global_stiffness_matrix(num_dof, ke_global, element_global_dof):
    K = np.zeros((num_dof, num_dof))
    for i in range(len(ke_global)):
        dof_indexing = element_global_dof[i]
        for row in range(1):
            for column in range(1):
                K[dof_indexing[row], dof_indexing[column]] += ke_global[i][row, column]
    return K

In [89]:
nodes = [[0.0, 0.0], [-10.0, -10.0], [10.0, -10.0]]
material_properties = [29000000.0, 10000.0, 15000000.0]
section_properties = [1.0, 10.0, 3.6]
elements = [[1, 2, 1, 2], [1, 3, 3, 1], [2, 3, 2, 3]]

A = [section_properties[elements[i][3] - 1] for i in range(len(elements))]
E = [material_properties[elements[i][2] - 1] for i in range(len(elements))]
L = [calculate_element_length(np.array(nodes[elements[i][0] - 1]), np.array(nodes[elements[i][1] - 1])) for i in range(len(elements))]

theta = [calculate_element_orientation(np.array(nodes[elements[i][0] - 1]), np.array(nodes[elements[i][1] - 1])) for i in range(len(elements))]

ke_local = [calculate_element_local_stiffness_matrix(E[i], A[i], L[i]) for i in range(len(L))]

beta = [define_element_rotation_matrix(theta[i]) for i in range(len(theta))]

ke_global = [define_global_element_stiffness_matrix(ke_local[i], beta[i]) for i in range(len(ke_local))]

element_global_dof = [define_global_dof_for_element(elements[i][0], elements[i][1]) for i in range(len(elements))]

num_dof = len(nodes) * 2

K = assemble_global_stiffness_matrix(num_dof, ke_global, element_global_dof)

print("Element Local Stiffness Matrices:")
print()
for i, ke in enumerate(ke_local):
    print(f"Element {i + 1} ke_local:{ke}")
    print()

print("Element Global Stiffness Matrices:")
print()
for i, ke in enumerate(ke_global):
    print(f"Element {i + 1} ke_global:{ke}")
    print()

print("Global Stiffness Matrix:")
print(K)

Element Local Stiffness Matrices:

Element 1 ke_local:[[ 20506096.65440988         0.         -20506096.65440988
          0.        ]
 [        0.                 0.                 0.
          0.        ]
 [-20506096.65440988         0.          20506096.65440988
          0.        ]
 [        0.                 0.                 0.
          0.        ]]

Element 2 ke_local:[[ 1060660.17177982        0.         -1060660.17177982        0.        ]
 [       0.                0.                0.                0.        ]
 [-1060660.17177982        0.          1060660.17177982        0.        ]
 [       0.                0.                0.                0.        ]]

Element 3 ke_local:[[ 1800.     0. -1800.     0.]
 [    0.     0.     0.     0.]
 [-1800.     0.  1800.     0.]
 [    0.     0.     0.     0.]]

Element Global Stiffness Matrices:

Element 1 ke_global:[[ 10253048.32720494  10253048.32720494 -10253048.32720494
  -10253048.32720494]
 [ 10253048.32720494  10253048.32